In [15]:
import csv
import math
import zipfile
import xml.etree.ElementTree as ET
from collections import defaultdict

"""
Hall & Jones (1999) Replication — Data Merge Pipeline
======================================================
Merges all datasets step by step, with checks at each stage.

Run this script from the folder where your data files are located,
or update the file paths in the CONFIG section below.

Steps:
  1. PWT 10.01        → output per worker, investment share, employment (1988)
  2. Capital stock    → constructed via perpetual inventory method within PWT
  3. Barro-Lee 2013   → average years of schooling (1985)
  4. Human capital    → computed from Barro-Lee using piecewise Mincerian returns
  5. Mining VA        → mining value added as % of GDP (1990 proxy for 1988)
  6. WGI governance   → composite index from rl, ge, cc (substitute for GADP)
  7. Sachs-Warner     → openness fraction (fraction of years open 1950-1992)
  8. Instruments      → distance from equator, FR trade, language fractions

Output: hj_master.csv — one row per country, ready for analysis
"""


# =============================================================================
# CONFIG — update these paths if needed
# =============================================================================
PWT_FILE        = "C:\\Users\\Adams\\OneDrive\\DE & E Research\\Data\\GDP, Investment, and Labor force\\pwt1001.xlsx"

BL_FILE         = "C:\\Users\\Adams\\OneDrive\\DE & E Research\\Data\\Barro-Lee dataset\\BL2013_MF2599_v2.csv"

MINING_FILE     = "C:\\Users\\Adams\\OneDrive\\DE & E Research\\Data\\MINING VALUE ADDED\\Contribution of mining to value added.xlsx"

WGI_FILE        = "C:\\Users\\Adams\\OneDrive\\DE & E Research\\Data\\For Index\\World Bank Governance Indicators\\wgidataset_with_sourcedata-2025.xlsx"

OPEN_FILE       = "C:\\Users\\Adams\\OneDrive\\DE & E Research\\Data\\For Index\\SACHS-WARNER OPENNESS INDEX\\open.csv"

GEO_FILE        = "C:\\Users\\Adams\\OneDrive\\DE & E Research\\Data\\INSTRUMENTAL Variables Data\\geo_instruments.csv"
FR_FILE         = "C:\\Users\\Adams\\OneDrive\\DE & E Research\\Data\\INSTRUMENTAL Variables Data\\frankel_romer_trade.csv"
LANG_FILE       = "C:\\Users\\Adams\\OneDrive\\DE & E Research\\Data\\INSTRUMENTAL Variables Data\\language_instruments.csv"

OUTPUT_FILE     = "C:\\Users\\Adams\\OneDrive\\DE & E Research\\outputs\\merged.csv"


In [16]:
# =============================================================================
# HELPER FUNCTIONS
# =============================================================================

def read_xlsx_sheet(filepath, sheet_index=0):
    """
    Read a sheet from an xlsx file.
    Returns (headers, rows) where rows is a list of dicts.
    sheet_index: 0-based index of the sheet to read.
    """
    with zipfile.ZipFile(filepath, 'r') as z:
        # Read shared strings
        with z.open('xl/sharedStrings.xml') as f:
            tree = ET.parse(f)
            root = tree.getroot()
            ns = {'ns': 'http://schemas.openxmlformats.org/spreadsheetml/2006/main'}
            strings = []
            for si in root.findall('ns:si', ns):
                t = si.find('ns:t', ns)
                if t is not None:
                    strings.append(t.text or '')
                else:
                    texts = si.findall('.//ns:t', ns)
                    strings.append(''.join(t.text or '' for t in texts))

        # Find sheet files in order
        sheet_files = sorted([
            n for n in z.namelist()
            if 'worksheets/sheet' in n and n.endswith('.xml')
        ])

        def get_val(cell, strings):
            t = cell.get('t')
            v = cell.find('{http://schemas.openxmlformats.org/spreadsheetml/2006/main}v')
            if v is None:
                return None
            if t == 's':
                idx = int(v.text)
                return strings[idx] if idx < len(strings) else None
            return v.text

        with z.open(sheet_files[sheet_index]) as f:
            tree = ET.parse(f)
            root = tree.getroot()
            rows = root.findall(
                './/{http://schemas.openxmlformats.org/spreadsheetml/2006/main}row'
            )

        if not rows:
            return [], []

        headers = [get_val(c, strings) for c in rows[0]]
        data = []
        for row in rows[1:]:
            vals = [get_val(c, strings) for c in row]
            d = dict(zip(headers, vals))
            data.append(d)

        return headers, data


def safe_float(val, default=None):
    """Convert a value to float, returning default if conversion fails."""
    if val is None or val == '':
        return default
    try:
        return float(val)
    except (ValueError, TypeError):
        return default


def safe_print(label, df):
    """Print a summary of the current dataset."""
    print(f"\n{'='*60}")
    print(f"  {label}")
    print(f"  Countries: {len(df)}")
    cols = list(df[list(df.keys())[0]].keys()) if df else []
    print(f"  Variables: {cols}")
    # Count missing values for key variables
    for key_var in ['yl', 'hc', 'csh_i', 'yr_sch', 'mining_va', 'governance', 'openness']:
        if df and key_var in list(df.values())[0]:
            n_missing = sum(1 for v in df.values() if v.get(key_var) is None)
            print(f"  Missing {key_var}: {n_missing}")
    print(f"{'='*60}")


In [17]:
# =============================================================================
# STEP 1: PWT 10.01 — Extract 1988 data
# =============================================================================
print("\n>>> STEP 1: Loading PWT 10.01...")

# PWT has 3 sheets: Info (index 0), Legend (index 1), Data (index 2)
# Based on our earlier inspection: sheet file order is sheet1=Info, sheet2=Legend, sheet3=Data
# But the workbook maps: Data=sheet id 1, Info=sheet id 2, Legend=sheet id 3
# The sheet files are sorted: sheet1.xml, sheet2.xml, sheet3.xml
# From our earlier inspection sheet3.xml = Data
_, pwt_rows = read_xlsx_sheet(PWT_FILE, sheet_index=2)

print(f"  PWT total rows loaded: {len(pwt_rows)}")

# Filter to year 1988
pwt_1988 = {}
for row in pwt_rows:
    if row.get('year') == '1988':
        iso = row.get('countrycode', '').strip()
        if not iso:
            continue

        rgdpo  = safe_float(row.get('rgdpo'))   # Output-side real GDP (mil 2017 USD PPP)
        emp    = safe_float(row.get('emp'))      # Employment (millions)
        pop    = safe_float(row.get('pop'))      # Population (millions)
        csh_i  = safe_float(row.get('csh_i'))    # Investment share of GDP
        hc_pwt = safe_float(row.get('hc'))       # PWT human capital index (backup)
        delta  = safe_float(row.get('delta'))    # Depreciation rate
        rkna   = safe_float(row.get('rkna'))     # Capital stock (nat. accounts, mil 2017 USD)
        labsh  = safe_float(row.get('labsh'))    # Labor share

        # Output per worker: Y/L
        yl = (rgdpo / emp) if (rgdpo and emp and emp > 0) else None

        pwt_1988[iso] = {
            'country':  row.get('country', ''),
            'iso3':     iso,
            'rgdpo':    rgdpo,
            'emp':      emp,
            'pop':      pop,
            'csh_i':    csh_i,
            'hc_pwt':   hc_pwt,
            'delta':    delta,
            'rkna':     rkna,
            'labsh':    labsh,
            'yl':       yl,        # Output per worker (main dependent variable)
        }

print(f"  Countries in 1988: {len(pwt_1988)}")
print(f"  Countries with Y/L: {sum(1 for v in pwt_1988.values() if v['yl'] is not None)}")

# Quick sanity check: US output per worker should be high
if 'USA' in pwt_1988:
    print(f"  USA Y/L check: {pwt_1988['USA']['yl']:.0f} (2017 USD PPP)")
if 'NER' in pwt_1988:
    print(f"  Niger Y/L check: {pwt_1988['NER']['yl']:.0f} (2017 USD PPP)")




>>> STEP 1: Loading PWT 10.01...
  PWT total rows loaded: 12810
  Countries in 1988: 183
  Countries with Y/L: 149
  USA Y/L check: 79440 (2017 USD PPP)
  Niger Y/L check: 2987 (2017 USD PPP)


In [18]:
# =============================================================================
# STEP 2: Capital Stock — Perpetual Inventory Method
# =============================================================================
print("\n>>> STEP 2: Constructing capital stocks via perpetual inventory...")

# Formula: K_t = I_t + (1 - delta) * K_{t-1}
# Initial value: K_0 = I_0 / (g + delta)
#   where g = average geometric growth rate of investment over first 10 years
# We use delta = 0.06 (Hall & Jones assumption)

#rgdpo = Output-side real GDP at chained PPPs (in mil. 2017US$) 
#csh_i = Investment share of GDP (unitless, e.g. 0.20 for 20%)
# Investment I_t = csh_i * rgdpo (investment in mil 2017 USD)

DELTA = 0.06  # depreciation rate

# Load all PWT years for capital stock construction
print("  Loading all PWT years for capital stock construction...")
all_pwt = defaultdict(dict)
for row in pwt_rows:
    iso = row.get('countrycode', '').strip()
    yr  = row.get('year', '').strip()
    if not iso or not yr:
        continue
    try:
        year = int(float(yr))
    except:
        continue
    rgdpo = safe_float(row.get('rgdpo'))
    csh_i = safe_float(row.get('csh_i'))
    # Investment = investment share * GDP
    inv = rgdpo * csh_i if (rgdpo and csh_i) else None
    all_pwt[iso][year] = inv

capital_stocks = {}
for iso, inv_series in all_pwt.items():
    years_with_data = sorted(y for y, v in inv_series.items() if v is not None)
    if not years_with_data:
        continue

    # Only proceed if we have data up to 1988
    if 1988 not in years_with_data and max(years_with_data) < 1988:
        continue

    first_year = years_with_data[0]

    # Compute initial growth rate from first 10 years of data
    early_years = [y for y in years_with_data if y <= first_year + 10]
    if len(early_years) >= 2:
        i_start = inv_series[early_years[0]]
        i_end   = inv_series[early_years[-1]]
        n_yrs   = early_years[-1] - early_years[0]
        if i_start and i_end and i_start > 0 and i_end > 0 and n_yrs > 0:
            g = (i_end / i_start) ** (1 / n_yrs) - 1
        else:
            g = 0.02  # default growth rate if data is problematic
    else:
        g = 0.02

    # Initial capital stock
    i0 = inv_series.get(first_year)
    if i0 is None or i0 <= 0:
        continue
    k = i0 / (g + DELTA)

    # Iterate forward to 1988
    for year in range(first_year + 1, 1989):
        inv = inv_series.get(year)
        if inv is not None and inv >= 0:
            k = inv + (1 - DELTA) * k
        else:
            # Missing year: just depreciate
            k = (1 - DELTA) * k

    capital_stocks[iso] = k

# Merge capital stock into master
for iso in pwt_1988:
    k = capital_stocks.get(iso)
    pwt_1988[iso]['K'] = k
    # Capital-output ratio K/Y (key input for production function)
    ky = (k / pwt_1988[iso]['rgdpo']) if (k and pwt_1988[iso].get('rgdpo')) else None
    pwt_1988[iso]['ky_ratio'] = ky

print(f"  Capital stocks constructed for: {sum(1 for v in pwt_1988.values() if v.get('K') is not None)} countries")
print(f"  USA K/Y ratio: {pwt_1988.get('USA', {}).get('ky_ratio', 'NA'):.3f}" if pwt_1988.get('USA', {}).get('ky_ratio') else "  USA K/Y: NA")



>>> STEP 2: Constructing capital stocks via perpetual inventory...
  Loading all PWT years for capital stock construction...
  Capital stocks constructed for: 183 countries
  USA K/Y ratio: 2.839


In [19]:
# =============================================================================
# STEP 3: Barro-Lee Education Data (1985)
# =============================================================================
print("\n>>> STEP 3: Loading Barro-Lee education data (1985)...")

bl_data = {}
with open(BL_FILE, 'r', encoding='latin1') as f:
    reader = csv.DictReader(f)
    for row in reader:
        # Use year=1985, sex=MF, age 25+
        if row.get('year') == '1985' and row.get('sex') == 'MF' and row.get('agefrom') == '25':
            iso = row.get('WBcode', '').strip()
            yr_sch = safe_float(row.get('yr_sch'))
            if iso:
                bl_data[iso] = {'yr_sch': yr_sch}

print(f"  Countries with 1985 education data: {len(bl_data)}")


# =============================================================================
# STEP 4: Human Capital Index (Piecewise Mincerian Returns)
# =============================================================================
print("\n>>> STEP 4: Computing human capital index from Barro-Lee...")

# Hall & Jones use piecewise Mincerian returns (Psacharopoulos 1994):
#   Years 1-4:  13.4% return per year (sub-Saharan Africa average)
#   Years 5-8:  10.1% return per year (world average)
#   Years 9+:    6.8% return per year (OECD average)
#
# Human capital index: h = exp(phi(E))
# phi(E) = 0.134*min(E,4) + 0.101*max(min(E,8)-4,0) + 0.068*max(E-8,0)

def mincerian_phi(E):
    """Compute piecewise Mincerian returns for E years of schooling."""
    if E is None:
        return None
    e1 = min(E, 4)
    e2 = max(min(E, 8) - 4, 0)
    e3 = max(E - 8, 0)
    return 0.134 * e1 + 0.101 * e2 + 0.068 * e3

for iso, d in bl_data.items():
    phi = mincerian_phi(d.get('yr_sch'))
    d['hc_bl'] = math.exp(phi) if phi is not None else None

print(f"  Human capital index computed for: {sum(1 for d in bl_data.values() if d.get('hc_bl') is not None)} countries")
print(f"  USA h check:   {bl_data.get('USA', {}).get('hc_bl', 'NA'):.4f}" if bl_data.get('USA', {}).get('hc_bl') else "  USA h: NA")
print(f"  Niger h check: {bl_data.get('NER', {}).get('hc_bl', 'NA'):.4f}" if bl_data.get('NER', {}).get('hc_bl') else "  Niger h: NA")

# Merge Barro-Lee into master
matched_bl = 0
for iso in pwt_1988:
    bl = bl_data.get(iso, {})
    pwt_1988[iso]['yr_sch'] = bl.get('yr_sch')
    pwt_1988[iso]['hc_bl']  = bl.get('hc_bl')
    # Use PWT hc as fallback if Barro-Lee missing
    if bl.get('hc_bl') is not None:
        pwt_1988[iso]['hc'] = bl.get('hc_bl')
        matched_bl += 1
    else:
        pwt_1988[iso]['hc'] = pwt_1988[iso].get('hc_pwt')  # PWT fallback

print(f"  Matched Barro-Lee to PWT: {matched_bl} countries")
print(f"  Using PWT hc as fallback for: {sum(1 for v in pwt_1988.values() if v.get('hc_bl') is None and v.get('hc_pwt') is not None)} countries")




>>> STEP 3: Loading Barro-Lee education data (1985)...
  Countries with 1985 education data: 146

>>> STEP 4: Computing human capital index from Barro-Lee...
  Human capital index computed for: 146 countries
  USA h check:   3.3923
  Niger h check: 1.0765
  Matched Barro-Lee to PWT: 138 countries
  Using PWT hc as fallback for: 5 countries


In [20]:
# =============================================================================
# STEP 5: Mining Value Added (1990 proxy for 1988)
# =============================================================================
print("\n>>> STEP 5: Loading mining value added (1990 values)...")

# The mining file uses country NAMES, not ISO codes.
# We need a crosswalk for the most common name differences.
NAME_TO_ISO = {
    'Afghanistan': 'AFG', 'Albania': 'ALB', 'Algeria': 'DZA', 'Angola': 'AGO',
    'Anguilla': 'AIA', 'Antigua and Barbuda': 'ATG', 'Argentina': 'ARG',
    'Armenia': 'ARM', 'Australia': 'AUS', 'Austria': 'AUT', 'Azerbaijan': 'AZE',
    'Bahamas': 'BHS', 'Bahrain': 'BHR', 'Bangladesh': 'BGD', 'Barbados': 'BRB',
    'Belarus': 'BLR', 'Belgium': 'BEL', 'Belize': 'BLZ', 'Benin': 'BEN',
    'Bhutan': 'BTN', 'Bolivia': 'BOL', 'Bosnia and Herzegovina': 'BIH',
    'Botswana': 'BWA', 'Brazil': 'BRA', 'Brunei Darussalam': 'BRN',
    'Bulgaria': 'BGR', 'Burkina Faso': 'BFA', 'Burundi': 'BDI',
    'Cabo Verde': 'CPV', 'Cambodia': 'KHM', 'Cameroon': 'CMR', 'Canada': 'CAN',
    'Central African Republic': 'CAF', 'Chad': 'TCD', 'Chile': 'CHL',
    'China': 'CHN', 'Colombia': 'COL', 'Comoros': 'COM', 'Congo': 'COG',
    'Costa Rica': 'CRI', 'Croatia': 'HRV', 'Cuba': 'CUB', 'Cyprus': 'CYP',
    'Czech Republic': 'CZE', 'Czechia': 'CZE',
    'Democratic Republic of the Congo': 'COD', 'Denmark': 'DNK',
    'Djibouti': 'DJI', 'Dominican Republic': 'DOM', 'Ecuador': 'ECU',
    'Egypt': 'EGY', 'El Salvador': 'SLV', 'Equatorial Guinea': 'GNQ',
    'Eritrea': 'ERI', 'Estonia': 'EST', 'Eswatini': 'SWZ', 'Ethiopia': 'ETH',
    'Fiji': 'FJI', 'Finland': 'FIN', 'France': 'FRA', 'Gabon': 'GAB',
    'Gambia': 'GMB', 'Georgia': 'GEO', 'Germany': 'DEU', 'Ghana': 'GHA',
    'Greece': 'GRC', 'Guatemala': 'GTM', 'Guinea': 'GIN',
    'Guinea-Bissau': 'GNB', 'Guyana': 'GUY', 'Haiti': 'HTI',
    'Honduras': 'HND', 'Hungary': 'HUN', 'Iceland': 'ISL', 'India': 'IND',
    'Indonesia': 'IDN', 'Iran': 'IRN', 'Iran (Islamic Republic of)': 'IRN',
    'Iraq': 'IRQ', 'Ireland': 'IRL', 'Israel': 'ISR', 'Italy': 'ITA',
    'Jamaica': 'JAM', 'Japan': 'JPN', 'Jordan': 'JOR', 'Kazakhstan': 'KAZ',
    'Kenya': 'KEN', 'Kuwait': 'KWT', 'Kyrgyzstan': 'KGZ', 'Lao PDR': 'LAO',
    'Latvia': 'LVA', 'Lebanon': 'LBN', 'Lesotho': 'LSO', 'Liberia': 'LBR',
    'Libya': 'LBY', 'Lithuania': 'LTU', 'Luxembourg': 'LUX',
    'Madagascar': 'MDG', 'Malawi': 'MWI', 'Malaysia': 'MYS',
    'Maldives': 'MDV', 'Mali': 'MLI', 'Malta': 'MLT', 'Mauritania': 'MRT',
    'Mauritius': 'MUS', 'Mexico': 'MEX', 'Moldova': 'MDA', 'Mongolia': 'MNG',
    'Morocco': 'MAR', 'Mozambique': 'MOZ', 'Myanmar': 'MMR', 'Namibia': 'NAM',
    'Nepal': 'NPL', 'Netherlands': 'NLD', 'New Zealand': 'NZL',
    'Nicaragua': 'NIC', 'Niger': 'NER', 'Nigeria': 'NGA', 'Norway': 'NOR',
    'Oman': 'OMN', 'Pakistan': 'PAK', 'Panama': 'PAN',
    'Papua New Guinea': 'PNG', 'Paraguay': 'PRY', 'Peru': 'PER',
    'Philippines': 'PHL', 'Poland': 'POL', 'Portugal': 'PRT', 'Qatar': 'QAT',
    'Romania': 'ROU', 'Russia': 'RUS', 'Russian Federation': 'RUS',
    'Rwanda': 'RWA', 'Saudi Arabia': 'SAU', 'Senegal': 'SEN',
    'Sierra Leone': 'SLE', 'Singapore': 'SGP', 'Slovakia': 'SVK',
    'Slovenia': 'SVN', 'Somalia': 'SOM', 'South Africa': 'ZAF',
    'South Korea': 'KOR', 'Spain': 'ESP', 'Sri Lanka': 'LKA', 'Sudan': 'SDN',
    'Suriname': 'SUR', 'Sweden': 'SWE', 'Switzerland': 'CHE', 'Syria': 'SYR',
    'Syrian Arab Republic': 'SYR', 'Taiwan': 'TWN', 'Tajikistan': 'TJK',
    'Tanzania': 'TZA', 'Thailand': 'THA', 'Togo': 'TGO',
    'Trinidad and Tobago': 'TTO', 'Tunisia': 'TUN', 'Turkey': 'TUR',
    'Turkmenistan': 'TKM', 'Uganda': 'UGA', 'Ukraine': 'UKR',
    'United Arab Emirates': 'ARE', 'United Kingdom': 'GBR',
    'United Republic of Tanzania': 'TZA', 'United States': 'USA',
    'United States of America': 'USA', 'Uruguay': 'URY', 'Uzbekistan': 'UZB',
    'Venezuela': 'VEN', 'Venezuela (Bolivarian Republic of)': 'VEN',
    'Viet Nam': 'VNM', 'Vietnam': 'VNM', 'Yemen': 'YEM', 'Zambia': 'ZMB',
    'Zimbabwe': 'ZWE', 'Zaire': 'ZAR', 'Ivory Coast': 'CIV',
    "Côte d'Ivoire": 'CIV', "Cote d'Ivoire": 'CIV',
    'Republic of Korea': 'KOR', 'Korea, Republic of': 'KOR',
    'Democratic People\'s Republic of Korea': 'PRK',
    'Swaziland': 'SWZ', 'Cape Verde': 'CPV',
    'Bolivia (Plurinational State of)': 'BOL',
    'Congo, Dem. Rep.': 'COD', 'Congo, Rep.': 'COG',
}

mining_data = {}
_, mining_rows = read_xlsx_sheet(MINING_FILE, sheet_index=1)  # sheet2 = Data

for row in mining_rows:
    country_name = row.get('Country and area', '').strip()
    if not country_name:
        continue
    iso = NAME_TO_ISO.get(country_name)
    if not iso:
        continue
    # Use 1990 as proxy for 1988
    val = safe_float(row.get('1990'))
    if val is not None:
        mining_data[iso] = val / 100  # convert % to fraction

print(f"  Mining data loaded for: {len(mining_data)} countries")

# Merge mining into master
for iso in pwt_1988:
    pwt_1988[iso]['mining_va'] = mining_data.get(iso)

matched_mining = sum(1 for v in pwt_1988.values() if v.get('mining_va') is not None)
print(f"  Matched mining data: {matched_mining} countries")
print(f"  Note: Using 1990 values as proxy for 1988 (2-year deviation, documented)")



>>> STEP 5: Loading mining value added (1990 values)...
  Mining data loaded for: 149 countries
  Matched mining data: 142 countries
  Note: Using 1990 values as proxy for 1988 (2-year deviation, documented)


In [21]:
# =============================================================================
# STEP 6: WGI Governance Index (substitute for GADP)
# =============================================================================
print("\n>>> STEP 6: Constructing WGI governance index...")

# Use Rule of Law (rl), Government Effectiveness (ge), Control of Corruption (cc)
# These are the three WGI dimensions closest to GADP components.
# We use the Governance score (0-100) column and average the three dimensions.
# Year: 1996 (earliest available — document as deviation from 1986-1995 GADP)

WGI_SHEETS = {
    'rl': 4,  # sheet index for rule of law
    'ge': 2,  # sheet index for govt effectiveness
    'cc': 5,  # sheet index for control of corruption
}
WGI_YEAR = '1996'

wgi_scores = defaultdict(dict)
for dim, sheet_idx in WGI_SHEETS.items():
    _, wgi_rows = read_xlsx_sheet(WGI_FILE, sheet_index=sheet_idx)
    for row in wgi_rows:
        if row.get('Year') != WGI_YEAR:
            continue
        iso = row.get('Economy (code)', '').strip()
        # Governance score 0-100
        score = safe_float(row.get('Governance score (0-100)'))
        if iso and score is not None:
            wgi_scores[iso][dim] = score

# Average across three dimensions, normalize to [0,1]
wgi_index = {}
for iso, dims in wgi_scores.items():
    available = [v for v in dims.values() if v is not None]
    if available:
        avg = sum(available) / len(available)
        wgi_index[iso] = avg / 100  # normalize to [0,1]

print(f"  WGI index computed for: {len(wgi_index)} countries")
print(f"  USA governance check:   {wgi_index.get('USA', 'NA'):.4f}" if wgi_index.get('USA') else "  USA WGI: NA")
print(f"  Niger governance check: {wgi_index.get('NER', 'NA'):.4f}" if wgi_index.get('NER') else "  Niger WGI: NA")

# Merge WGI into master
for iso in pwt_1988:
    pwt_1988[iso]['governance'] = wgi_index.get(iso)

matched_wgi = sum(1 for v in pwt_1988.values() if v.get('governance') is not None)
print(f"  Matched WGI data: {matched_wgi} countries")



>>> STEP 6: Constructing WGI governance index...
  WGI index computed for: 199 countries
  USA governance check:   0.8348
  Niger governance check: 0.3433
  Matched WGI data: 175 countries


In [22]:
# =============================================================================
# STEP 7: Sachs-Warner Openness (fraction of years open)
# =============================================================================
print("\n>>> STEP 7: Computing Sachs-Warner openness fractions...")

# open.csv has annual binary OPEN variable (0/1) per country per year
# We compute fraction of years open over available data (1950-1992)
# Hall & Jones use 1950-1994, but our data ends in 1992 — documented deviation

# Country name to ISO crosswalk for Sachs-Warner (uses country names)
SW_NAME_TO_ISO = {
    'ALGERIA': 'DZA', 'BENIN': 'BEN', 'BOTSWANA': 'BWA', 'BURKINA FASO': 'BFA',
    'BURUNDI': 'BDI', 'CAMEROON': 'CMR', 'CAPE VERDE': 'CPV', 'CENTRAL AFRICA': 'CAF',
    'CHAD': 'TCD', 'COMOROS': 'COM', 'CONGO': 'COG', 'DJIBOUTI': 'DJI',
    'EGYPT': 'EGY', 'ETHIOPIA': 'ETH', 'GABON': 'GAB', 'GAMBIA': 'GMB',
    'GHANA': 'GHA', 'GUINEA': 'GIN', 'GUINEA-BISSAU': 'GNB',
    'IVORY COAST': 'CIV', 'KENYA': 'KEN', 'LESOTHO': 'LSO', 'LIBERIA': 'LBR',
    'MADAGASCAR': 'MDG', 'MALAWI': 'MWI', 'MALI': 'MLI', 'MAURITANIA': 'MRT',
    'MAURITIUS': 'MUS', 'MOROCCO': 'MAR', 'MOZAMBIQUE': 'MOZ', 'NAMIBIA': 'NAM',
    'NIGER': 'NER', 'NIGERIA': 'NGA', 'RWANDA': 'RWA', 'SENEGAL': 'SEN',
    'SEYCHELLES': 'SYC', 'SIERRA LEONE': 'SLE', 'SOMALIA': 'SOM',
    'SOUTH AFRICA': 'ZAF', 'SUDAN': 'SDN', 'SWAZILAND': 'SWZ',
    'TANZANIA': 'TZA', 'TOGO': 'TGO', 'TUNISIA': 'TUN', 'UGANDA': 'UGA',
    'ZAIRE': 'ZAR', 'ZAMBIA': 'ZMB', 'ZIMBABWE': 'ZWE',
    'BAHAMAS': 'BHS', 'BARBADOS': 'BRB', 'BELIZE': 'BLZ', 'CANADA': 'CAN',
    'COSTA RICA': 'CRI', 'DOMINICA': 'DMA', 'DOMINICAN REPUBLIC': 'DOM',
    'EL SALVADOR': 'SLV', 'GRENADA': 'GRD', 'GUATEMALA': 'GTM',
    'HAITI': 'HTI', 'HONDURAS': 'HND', 'JAMAICA': 'JAM', 'MEXICO': 'MEX',
    'NICARAGUA': 'NIC', 'PANAMA': 'PAN', 'PUERTO RICO': 'PRI',
    'TRINIDAD AND TOBAGO': 'TTO', 'TRINIDAD & TOBAGO': 'TTO',
    'UNITED STATES': 'USA', 'U.S.A.': 'USA', 'U.S.S.R.': 'SUN',
    'BRAZIL': 'BRA', 'CHILE': 'CHL', 'COLOMBIA': 'COL', 'ECUADOR': 'ECU',
    'GUYANA': 'GUY', 'PARAGUAY': 'PRY', 'PERU': 'PER', 'SURINAME': 'SUR',
    'URUGUAY': 'URY', 'VENEZUELA': 'VEN', 'BAHRAIN': 'BHR',
    'BANGLADESH': 'BGD', 'BHUTAN': 'BTN', 'CHINA': 'CHN', 'HONG KONG': 'HKG',
    'INDIA': 'IND', 'INDONESIA': 'IDN', 'IRAN': 'IRN', 'IRAQ': 'IRQ',
    'ISRAEL': 'ISR', 'JAPAN': 'JPN', 'JORDAN': 'JOR', 'KOREA': 'KOR',
    'KUWAIT': 'KWT', 'LAOS': 'LAO', 'MALAYSIA': 'MYS', 'MONGOLIA': 'MNG',
    'MYANMAR': 'MMR', 'NEPAL': 'NPL', 'OMAN': 'OMN', 'PAKISTAN': 'PAK',
    'PHILIPPINES': 'PHL', 'QATAR': 'QAT', 'SAUDI ARABIA': 'SAU',
    'SINGAPORE': 'SGP', 'SRI LANKA': 'LKA', 'SYRIA': 'SYR', 'TAIWAN': 'TWN',
    'THAILAND': 'THA', 'UNITED ARAB EMIRATES': 'ARE', 'YEMEN': 'YEM',
    'AUSTRIA': 'AUT', 'BELGIUM': 'BEL', 'BULGARIA': 'BGR', 'CYPRUS': 'CYP',
    'CZECHOSLOVAKIA': 'CZE', 'DENMARK': 'DNK', 'FINLAND': 'FIN',
    'FRANCE': 'FRA', 'GERMANY': 'DEU', 'WEST GERMANY': 'DEU',
    'GREECE': 'GRC', 'HUNGARY': 'HUN', 'ICELAND': 'ISL', 'IRELAND': 'IRL',
    'ITALY': 'ITA', 'LUXEMBOURG': 'LUX', 'MALTA': 'MLT',
    'NETHERLANDS': 'NLD', 'NORWAY': 'NOR', 'POLAND': 'POL',
    'PORTUGAL': 'PRT', 'ROMANIA': 'ROU', 'SPAIN': 'ESP', 'SWEDEN': 'SWE',
    'SWITZERLAND': 'CHE', 'TURKEY': 'TUR', 'UNITED KINGDOM': 'GBR',
    'SOVIET UNION': 'SUN', 'YUGOSLAVIA': 'YUG', 'AUSTRALIA': 'AUS',
    'FIJI': 'FJI', 'NEW ZEALAND': 'NZL', 'PAPUA NEW GUINEA': 'PNG',
}

open_years = defaultdict(lambda: {'open': 0, 'total': 0})
with open(OPEN_FILE, 'r', encoding='latin1') as f:
    reader = csv.DictReader(f)
    for row in reader:
        country = row.get('COUNTRY', '').strip().upper()
        iso = SW_NAME_TO_ISO.get(country)
        if not iso:
            continue
        open_val = row.get('OPEN')
        if open_val is None:
            continue
        open_val = str(open_val).strip()
        if open_val in ('0.00', '1.00', '0', '1'):
            open_years[iso]['total'] += 1
            if open_val in ('1.00', '1'):
                open_years[iso]['open'] += 1

openness_fracs = {}
for iso, counts in open_years.items():
    if counts['total'] > 0:
        openness_fracs[iso] = counts['open'] / counts['total']

print(f"  Openness fractions computed for: {len(openness_fracs)} countries")
print(f"  USA openness check:   {openness_fracs.get('USA', 'NA'):.3f}" if 'USA' in openness_fracs else "  USA: NA")
print(f"  Niger openness check: {openness_fracs.get('NER', 'NA'):.3f}" if 'NER' in openness_fracs else "  Niger: NA")
print(f"  Switzerland check:    {openness_fracs.get('CHE', 'NA'):.3f}" if 'CHE' in openness_fracs else "  CHE: NA")

# Merge openness into master
for iso in pwt_1988:
    pwt_1988[iso]['openness'] = openness_fracs.get(iso)

matched_open = sum(1 for v in pwt_1988.values() if v.get('openness') is not None)
print(f"  Matched openness data: {matched_open} countries")




>>> STEP 7: Computing Sachs-Warner openness fractions...
  Openness fractions computed for: 99 countries
  USA openness check:   1.000
  Niger openness check: 0.000
  Switzerland check:    1.000
  Matched openness data: 96 countries


In [23]:
# =============================================================================
# STEP 8: Instruments — Distance from Equator, FR Trade, Language
# =============================================================================
print("\n>>> STEP 8: Merging instruments...")

# Distance from equator
geo_data = {}
with open(GEO_FILE, 'r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        iso = row.get('iso3', '').strip()
        dist = safe_float(row.get('distancefromeq'))
        if iso:
            geo_data[iso] = dist

# Frankel-Romer predicted trade share
fr_data = {}
with open(FR_FILE, 'r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        iso = row.get('iso3', '').strip()
        fr_val = safe_float(row.get('fr_constructed_trade_share'))
        if iso:
            fr_data[iso] = fr_val

# Language fractions
lang_data = {}
with open(LANG_FILE, 'r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        iso = row.get('iso3', '').strip()
        eng = safe_float(row.get('english_frac'))
        we  = safe_float(row.get('we_lang_frac'))
        if iso:
            lang_data[iso] = {'english_frac': eng, 'we_lang_frac': we}

# Merge all instruments
for iso in pwt_1988:
    pwt_1988[iso]['distancefromeq'] = geo_data.get(iso)
    pwt_1988[iso]['fr_trade']       = fr_data.get(iso)
    pwt_1988[iso]['english_frac']   = lang_data.get(iso, {}).get('english_frac')
    pwt_1988[iso]['we_lang_frac']   = lang_data.get(iso, {}).get('we_lang_frac')

print(f"  Distance from equator matched: {sum(1 for v in pwt_1988.values() if v.get('distancefromeq') is not None)}")
print(f"  FR trade matched:              {sum(1 for v in pwt_1988.values() if v.get('fr_trade') is not None)}")
print(f"  Language data matched:         {sum(1 for v in pwt_1988.values() if v.get('english_frac') is not None)}")



>>> STEP 8: Merging instruments...
  Distance from equator matched: 168
  FR trade matched:              139
  Language data matched:         166


In [24]:
# =============================================================================
# STEP 9: Construct Social Infrastructure Index
# =============================================================================
print("\n>>> STEP 9: Constructing social infrastructure index...")

# S = (governance + openness) / 2
# Both already on [0,1] scale (governance normalized from 0-100, openness is a fraction)
# This mirrors Hall & Jones: S = (GADP + Sachs-Warner) / 2

for iso, d in pwt_1988.items():
    gov = d.get('governance')
    opn = d.get('openness')
    if gov is not None and opn is not None:
        d['social_infra'] = (gov + opn) / 2
    elif gov is not None:
        d['social_infra'] = None  # require both components
    elif opn is not None:
        d['social_infra'] = None
    else:
        d['social_infra'] = None

n_si = sum(1 for v in pwt_1988.values() if v.get('social_infra') is not None)
print(f"  Social infrastructure index computed for: {n_si} countries")

# Quick check: Switzerland and US should be near top, Niger near bottom
for check_iso in ['USA', 'CHE', 'NER', 'ZAR']:
    si = pwt_1988.get(check_iso, {}).get('social_infra')
    print(f"  {check_iso} S = {si:.4f}" if si is not None else f"  {check_iso} S = NA")




>>> STEP 9: Constructing social infrastructure index...
  Social infrastructure index computed for: 96 countries
  USA S = 0.9174
  CHE S = 0.9471
  NER S = 0.1716
  ZAR S = NA


In [25]:
# =============================================================================
# STEP 10: Final Assembly and Quality Check
# =============================================================================
print("\n>>> STEP 10: Final assembly and quality check...")

# Define the complete set of variables in the output
OUTPUT_VARS = [
    'iso3', 'country',
    # Core variables
    'yl',           # output per worker (Y/L) — main dependent variable
    'rgdpo',        # real GDP (mil 2017 USD PPP)
    'emp',          # employment (millions)
    'pop',          # population (millions)
    # Production function components
    'K',            # capital stock
    'ky_ratio',     # capital-output ratio K/Y
    'hc',           # human capital index (Mincerian, from Barro-Lee; PWT fallback)
    'hc_bl',        # human capital from Barro-Lee specifically
    'hc_pwt',       # human capital from PWT (for comparison)
    'yr_sch',       # average years of schooling (Barro-Lee 1985)
    # Natural resource correction
    'mining_va',    # mining value added as fraction of GDP
    # Social infrastructure
    'governance',   # WGI composite (rl+ge+cc avg, normalized 0-1)
    'openness',     # Sachs-Warner fraction of years open
    'social_infra', # S = (governance + openness) / 2
    # Instruments
    'distancefromeq',  # |latitude| / 90
    'fr_trade',        # Frankel-Romer constructed trade share
    'english_frac',    # fraction speaking English natively
    'we_lang_frac',    # fraction speaking Western European language natively
    # Auxiliary
    'delta',           # depreciation rate
    'labsh',           # labor share (for checking alpha assumption)
    'csh_i',           # investment share of GDP
]



>>> STEP 10: Final assembly and quality check...


In [26]:
# Count complete cases (all core variables present)
core_vars = ['yl', 'ky_ratio', 'hc', 'social_infra']
complete = {
    iso: d for iso, d in pwt_1988.items()
    if all(d.get(v) is not None for v in core_vars)
}
print(f"  Complete cases (yl + K/Y + hc + S all present): {len(complete)}")
print(f"  Total countries in dataset: {len(pwt_1988)}")

# Countries with Y/L only (for partial replication)
has_yl = {iso: d for iso, d in pwt_1988.items() if d.get('yl') is not None}
print(f"  Countries with Y/L data: {len(has_yl)}")

# Missing data summary
print("\n  Missing data summary:")
for var in ['yl', 'ky_ratio', 'hc', 'yr_sch', 'mining_va', 'governance', 'openness', 'social_infra', 'distancefromeq', 'fr_trade']:
    n_missing = sum(1 for d in pwt_1988.values() if d.get(var) is None)
    print(f"    {var:<20}: {n_missing:>3} missing out of {len(pwt_1988)}")


  Complete cases (yl + K/Y + hc + S all present): 93
  Total countries in dataset: 183
  Countries with Y/L data: 149

  Missing data summary:
    yl                  :  34 missing out of 183
    ky_ratio            :  26 missing out of 183
    hc                  :  40 missing out of 183
    yr_sch              :  45 missing out of 183
    mining_va           :  41 missing out of 183
    governance          :   8 missing out of 183
    openness            :  87 missing out of 183
    social_infra        :  87 missing out of 183
    distancefromeq      :  15 missing out of 183
    fr_trade            :  44 missing out of 183


In [27]:
# =============================================================================
# STEP 11: Save Output
# =============================================================================
print(f"\n>>> STEP 11: Saving master dataset to {OUTPUT_FILE}...")

with open(OUTPUT_FILE, 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=OUTPUT_VARS, extrasaction='ignore')
    writer.writeheader()
    for iso in sorted(pwt_1988.keys()):
        row = {var: pwt_1988[iso].get(var, '') for var in OUTPUT_VARS}
        # Round floats to 6 decimal places
        for var in OUTPUT_VARS:
            val = row[var]
            if isinstance(val, float):
                row[var] = round(val, 6)
        writer.writerow(row)

print(f"  Saved {len(pwt_1988)} countries to {OUTPUT_FILE}")




>>> STEP 11: Saving master dataset to C:\Users\Adams\OneDrive\DE & E Research\outputs\merged.csv...
  Saved 183 countries to C:\Users\Adams\OneDrive\DE & E Research\outputs\merged.csv


In [28]:
# =============================================================================
# STEP 12: Verification Table — Reproduce key rows from Hall & Jones Table I
# =============================================================================
print("\n>>> STEP 12: Verification against Hall & Jones Table I...")

# Expected approximate values from paper (scaled to US = 1.0)
# Note: exact values will differ because we use updated data and WGI instead of GADP
verify_countries = {
    'USA': {'rank_yl': 1},
    'CAN': {},
    'GBR': {},
    'FRA': {},
    'DEU': {},
    'JPN': {},
    'BRA': {},
    'CHN': {},
    'IND': {},
    'KEN': {},
    'NGA': {},
    'NER': {},
    'ZAR': {},
}




>>> STEP 12: Verification against Hall & Jones Table I...


In [29]:
# Get USA values for normalization
usa = pwt_1988.get('USA', {})
usa_yl    = usa.get('yl', 1)
usa_ky    = usa.get('ky_ratio', 1)
usa_hc    = usa.get('hc', 1)

print(f"\n  {'ISO':<6} {'Y/L (rel USA)':>14} {'K/Y (rel USA)':>14} {'h (rel USA)':>12} {'S':>8}")
print(f"  {'-'*60}")

for iso in ['USA', 'CAN', 'GBR', 'JPN', 'FRA', 'DEU', 'BRA', 'CHN', 'IND', 'KEN', 'NGA', 'NER']:
    d = pwt_1988.get(iso, {})
    yl  = d.get('yl')
    ky  = d.get('ky_ratio')
    hc  = d.get('hc')
    si  = d.get('social_infra')

    rel_yl = f"{yl/usa_yl:.3f}"   if (yl and usa_yl) else 'NA'
    rel_ky = f"{ky/usa_ky:.3f}"   if (ky and usa_ky) else 'NA'
    rel_hc = f"{hc/usa_hc:.3f}"   if (hc and usa_hc) else 'NA'
    si_str = f"{si:.3f}"          if si is not None  else 'NA'

    print(f"  {iso:<6} {rel_yl:>14} {rel_ky:>14} {rel_hc:>12} {si_str:>8}")

print("\n  Note: Values differ from original paper because:")
print("  (1) PWT 10.01 uses 2017 prices, original used PWT 5.6 with 1985 prices")
print("  (2) WGI (1996) used instead of GADP (1986-1995)")
print("  (3) Capital stock constructed from updated investment data")
print("  These deviations should be documented in the replication report.")

print("\n>>> PIPELINE COMPLETE. Master dataset saved to:", OUTPUT_FILE)
print("    Next step: Run Hall & Jones analysis (levels accounting + IV regression)")


  ISO     Y/L (rel USA)  K/Y (rel USA)  h (rel USA)        S
  ------------------------------------------------------------
  USA             1.000          1.000        1.000    0.917
  CAN             0.874          0.940        0.855    0.907
  GBR             0.684          0.962        0.775       NA
  JPN             0.631          1.152        0.822    0.738
  FRA             0.803          1.134        0.656    0.793
  DEU             0.629          1.221        0.721       NA
  BRA             0.204          0.715        0.477    0.278
  CHN             0.059          0.663        0.545    0.224
  IND             0.042          0.628        0.406    0.250
  KEN             0.077          0.377        0.446    0.275
  NGA             0.043          7.099        0.359    0.154
  NER             0.038          0.882        0.317    0.172

  Note: Values differ from original paper because:
  (1) PWT 10.01 uses 2017 prices, original used PWT 5.6 with 1985 prices
  (2) WGI (1996) u